In [2]:
import pandas as pd
import numpy as np
import requests
from sshtunnel import SSHTunnelForwarder
import psycopg2
from tqdm import tqdm
tqdm.pandas()
#import distance as dst

In [140]:
hotel = pd.read_excel('./data/2019_hotel_data_주소_위경도.xlsx') # 호텔 표본

In [4]:
def find_sido(table):
    sido_list= table['지번주소'].split()
    return sido_list[0]

In [141]:
hotel['시도'] =  hotel.progress_apply(lambda x : find_sido(x),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 68112.69it/s]


In [6]:
def find_sigungu(table):
    if table['시도'] == '세종특별자치시':
        pass
    else: 
        sigungu_list= table['지번주소'].split()
        if sigungu_list[2][-1] =='구':
            return sigungu_list[1]+' '+sigungu_list[2]
        else:
            return sigungu_list[1]

In [142]:
hotel['시군구'] =  hotel.progress_apply(lambda x : find_sigungu(x),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 43049.41it/s]


In [143]:
bunji_list = []
def find_bjdong(table):
    
    try:
        bjdong_list= table['지번주소'].split()
        if table['시도'] == '세종특별자치시':
            if len(bjdong_list) == 2:
                bunji_list.append(bjdong_list[2])
                return bjdong_list[1]
            elif len(bjdong_list) == 3:
                bunji_list.append(bjdong_list[3])
                return bjdong_list[1]+' '+bjdong_list[2]
        else:
            if len(bjdong_list) == 4:
                bunji_list.append(bjdong_list[3])
                return bjdong_list[2]
            else:
                if any(chr.isdigit() for chr in bjdong_list[3]):
                        bunji_list.append(bjdong_list[3])
                        return bjdong_list[2]
                else:
                     if bjdong_list[2][-1] =='구':
                        bunji_list.append(bjdong_list[4])
                        return bjdong_list[3]
                     else:
                        bunji_list.append(bjdong_list[4])
                        return bjdong_list[2]+' '+bjdong_list[3]

    except:
        pass

In [144]:
hotel['법정동'] =  hotel.progress_apply(lambda x : find_bjdong(x),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 55887.07it/s]


In [145]:
hotel['번지']= bunji_list

In [11]:
def find_bun(table):
    try:
        return str(table.번지.split('-')[0]).zfill(4)
        
    except:
        return str('0000').zfill(4)

In [146]:
hotel['번'] = hotel.progress_apply(lambda x : find_bun(x),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 48353.26it/s]


In [13]:
def find_ji(table):
    try:
        return str(table.번지.split('-')[1]).zfill(4)
        
    except:
        return str('0000').zfill(4)

In [147]:
hotel['지'] = hotel.progress_apply(lambda x : find_ji(x),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 50848.10it/s]


In [15]:
admin = pd.read_excel('./data/법정동_20211117.xlsx') #법정동 코드 맴핑을 위한 법정동파일 출처 : 행정표준코드관리시스템

In [16]:
def find_bjdong_code(table):
    try:
        if table['시도'] =='세종특별자치시':
            sido = admin[admin.시도명 == '세종특별자치시']
            bjdong_list = table.법정동.split()
            if len(bjdong_list) == 1:
                return sido[sido.읍면동명 ==table['법정동']].법정동코드.values[0]
            elif len(bjdong_list) == 2:
                sigungu= sido[sido.읍면동명 ==bjdong_list[0]]
                return sigungu[sigungu.동리명==bjdong_list[1]].법정동코드.values[0]
        
        
        sido = admin[admin.시도명 == table['시도']]
        sigungu = sido[sido.시군구명==table['시군구']]
        bjdong_list = table.법정동.split()
        #len(bjdong_list)
        if len(bjdong_list) == 1:
            return sigungu[sigungu.읍면동명 ==table['법정동']].법정동코드.values[0]
        elif len(bjdong_list) == 2:
            hjdong_list= sigungu[sigungu.읍면동명 ==bjdong_list[0]]
            return hjdong_list[hjdong_list.동리명==bjdong_list[1]].법정동코드.values[0]
    except:
        pass

In [148]:
hotel['bjdong_code'] = hotel.progress_apply(lambda x : find_bjdong_code(x),axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:03<00:00, 491.26it/s]


In [239]:
hotel[hotel['bjdong_code'].isna()]

,Unnamed: 0,지번주소,도로명주소,위도,경도,번호,지역1(시도),지역2(시군구),업종,호텔명(등록명칭),...,주소(도로명주소),전화번호,부대시설,시도,시군구,법정동,번지,번,지,bjdong_code
1554,1555,서울특별시 중구 을지로 16,서울특별시 중구 을지로1가 188-3 백남빌딩,37.56561,126.979413,1556,서울특별시,중구,관광호텔,프레지던트호텔,...,NaN,NaN,NaN,서울특별시,중구,을지로,16,0016,0000,NaN


In [243]:
hotel[hotel['bjdong_code'].isna()].index

Int64Index([], dtype='int64')

In [242]:
hotel.loc[1554]

Unnamed: 0                                 1555
지번주소                           서울특별시 중구 을지로 16 
도로명주소                 서울특별시 중구 을지로1가 188-3 백남빌딩
위도                                     37.56561
경도                                   126.979413
번호                                         1556
지역1(시도)                                   서울특별시
지역2(시군구)                                     중구
업종                                         관광호텔
호텔명(등록명칭)                               프레지던트호텔
관광사업자명(법인명 or 개인명)                        백남관광㈜
영업상태(19.12.31기준)                            영업중
호텔등급                                         4성
등급부여일                       2018-03-14 00:00:00
결정기관                                     한국관광공사
층수(지하~지상)             지하3-지상2\n/지상7-27\n(10층제외)
대지면적(㎡)                                     NaN
건축연면적(㎡)                               24970.16
객실수                                         303
승인관리번호                                      NaN
최초승인일자                                  

In [241]:
hotel['bjdong_code'][1554] = '1114010400'

C:\Users\LEESOY~1\AppData\Local\Temp/ipykernel_20128/2200700428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel['bjdong_code'][1554] = '1114010400'


In [244]:
def make_pnu(table):
        return str(int(table['bjdong_code']))+'-0-'+str(table['번']).zfill(4)+str(table['지']).zfill(4)

In [245]:
hotel['PNU'] = hotel.progress_apply(lambda x : make_pnu(x), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1914/1914 [00:00<00:00, 52849.54it/s]


In [246]:
hotel.to_excel('./export_data/2019_hotel_data_PNU.xlsx',index=False, encoding='utf-8-sig')